In [ ]:
import pandas as pd

players = pd.read_csv('basketballPlayoffs/players.csv', delimiter=",")
coaches = pd.read_csv('basketballPlayoffs/coaches.csv', delimiter=",")
teams = pd.read_csv('basketballPlayoffs/teams.csv', delimiter=",")
players_teams = pd.read_csv('basketballPlayoffs/players_teams.csv', delimiter=",")
teams_post = pd.read_csv('basketballPlayoffs/teams_post.csv', delimiter=",")
series_post = pd.read_csv('basketballPlayoffs/series_post.csv', delimiter=",")
awards_players = pd.read_csv('basketballPlayoffs/awards_players.csv', delimiter=",")

print(players.head())
print(coaches)
print(teams)
print(players_teams)
print(teams_post)
print(series_post)
print(awards_players)


players.isnull().sum()
#players.describe()

In [ ]:
# to delete:  deathDate, collegeOther, , firstseason, lastseason linhas com pos=""
# show all players that have Pos: nullpd.set_option('display.max_rows', None)
print(players.isna().sum())
players = players.drop(columns=['collegeOther', 'deathDate', 'firstseason', 'lastseason'])
players = players[players['pos'] != ""]
print(players.head())

In [ ]:
# to delete: lgID 
print(coaches.isna().sum())
coaches = coaches.drop(columns=['lgID'])
print(coaches.head())

In [ ]:
# to delete: lgID, divID, seeded, tmORB, tmDRB, tmTRB, opptmORB, opptmDRB, opptmTRB
print(teams.isna().sum())   
teams = teams.drop(columns=['lgID', 'divID', 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB'])
print(teams.head())

In [ ]:
# to delete: lgID 
#print(players_teams.isna().sum())
players_teams = players_teams.drop(columns=['lgID'])
print(players_teams.head())

In [ ]:
# to delete: lgID
#print(teams_post.isna().sum())
teams_post = teams_post.drop(columns=['lgID'])
print(teams_post.head())

In [ ]:
# to delete: lgIDWinner, lgIDLoser
#print(series_post.isna().sum())
series_post = series_post.drop(columns=['lgIDWinner', 'lgIDLoser'])
print(series_post.head())

In [ ]:
# to delete: lgID
#print(awards_players.isna().sum()) 
awards_players = awards_players.drop(columns=['lgID'])
print(awards_players.head())